In [2]:
import theano
import theano.tensor as T
import numpy as np

In [3]:
lx, ly = T.dscalars('lx', 'ly')
N = T.iscalar('N')
delta_x = lx / N
iseq = T.arange(N)
fseq = T.dvector('fseq')

In [4]:
functional_1stterm = delta_x * T.sqrt(0.5*(1+((fseq[0]-ly)/(2*delta_x))**2)/(ly-fseq[0]))
functional_Nthterm = delta_x * T.sqrt(0.5*(1+((fseq[N-1]-fseq[N-2])/delta_x)**2)/(ly-fseq[N-1]))
functional_ithterm = lambda i: delta_x * T.sqrt(0.5*(1+((fseq[i+1]-fseq[i-1])/(2*delta_x))**2)/(ly-fseq[i]))
pfunc = lambda k: T.switch(T.eq(k, N-1), 
                           functional_Nthterm,
                           T.switch(T.eq(k, 0), 
                                    functional_1stterm,
                                    functional_ithterm(k)))

In [5]:
# functional_parts = pfunc(iseq)
functional_parts, _ = theano.map(fn=lambda k: pfunc(k), sequences=[iseq])
functional = functional_parts.sum()

In [6]:
time_brach = theano.function(inputs=[fseq, lx, ly, N], outputs=functional_parts)

/Users/hok1/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [9]:
j = T.iscalar('j')
pf = theano.function(inputs=[fseq, lx, ly, N, j], outputs=pfunc(j))
iseqfcn = theano.function(inputs=[N], outputs=iseq)
f1 = theano.function(inputs=[fseq, lx, ly, N], outputs=functional_1stterm)
fN = theano.function(inputs=[fseq, lx, ly, N], outputs=functional_Nthterm)
fi = theano.function(inputs=[fseq, lx, ly, N, j], outputs=functional_ithterm(j))

In [8]:
time_brach(np.array([0.75, 0.5, 0.25, 0.]), 1., 1., 4)

IndexError: index out of bounds
Apply node that caused the error: Subtensor{int32}(fseq_copy, ScalarFromTensor.0)
Inputs types: [TensorType(float64, vector), Scalar(int32)]

Backtrace when the node is created:
  File "<ipython-input-4-452e67e0891c>", line 3, in <lambda>
    functional_ithterm = lambda i: delta_x * T.sqrt(0.5*(1+((fseq[i+1]-fseq[i-1])/(2*delta_x))**2)/(ly-fseq[i]))

HINT: Use another linker then the c linker to have the inputs shapes and strides printed.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.
Apply node that caused the error: for{cpu,scan_fn}(Elemwise{Cast{int64}}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Elemwise{eq,no_inplace}.0, Elemwise{eq,no_inplace}.0, Subtensor{int64:int64:int8}.0, Elemwise{Cast{int64}}.0, fseq, ly, Elemwise{true_div,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{Composite{(i0 * sqrt(((i1 * (i2 + sqr(((i1 * (i3 - i4) * i5) / i6)))) / (i4 - i3))))}}.

In [13]:
pf(np.array([0.75, 0.5, 0.25, 0.]), 1., 1., 4,3)

IndexError: index out of bounds
Apply node that caused the error: Subtensor{int32}(fseq, ScalarFromTensor.0)
Inputs types: [TensorType(float64, vector), Scalar(int32)]
Inputs shapes: [(4,), ()]
Inputs strides: [(8,), ()]
Inputs values: [array([ 0.75,  0.5 ,  0.25,  0.  ]), 4]

Backtrace when the node is created:
  File "<ipython-input-4-452e67e0891c>", line 3, in <lambda>
    functional_ithterm = lambda i: delta_x * T.sqrt(0.5*(1+((fseq[i+1]-fseq[i-1])/(2*delta_x))**2)/(ly-fseq[i]))

HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [23]:
f1 = lambda i: T.switch(T.eq(i, N-1), 2*(N-1), T.switch(T.eq(i, 0), -1, i))
f1fcn = theano.function(inputs=[j, N], outputs=f1(j))

In [141]:
f1fcn(np.array([0,1]), 10)

TypeError: ('Bad input argument to theano function with name "<ipython-input-23-74c003c1e6ef>:2"  at index 0(0-based)', 'TensorType(int32, scalar) cannot store a value of dtype int64 without risking loss of precision. If you do not mind this loss, you can: 1) explicitly cast your data to int32, or 2) set "allow_input_downcast=True" when calling "function".', array([0, 1]))

In [155]:
test1, _ = theano.map(fn=lambda k: 2*k, sequences=[iseq])
test1fcn = theano.function(inputs=[N], outputs=test1)

In [156]:
test1fcn(10)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=int32)

In [15]:
iseq.size

Prod{acc_dtype=int64}.0